# Neural Net From Scratch

## Imports

In [2]:
import numpy as np
from random import random, seed

from sklearn.metrics import r2_score, accuracy_score

seed(0)

## Model

### Activation functions

In [28]:
class Linear():
    """Linear layer: y = W * I + B -> geen activatie functie, dus geef antwoord gewoon terug"""
    @staticmethod
    def f(x):
        return x
    @staticmethod
    def df(x):
        return 1

class Relu():
    @staticmethod
    def f(x):
        return x * (x > 0)
    @staticmethod
    def df(x):
        return 1 * (x > 0)

class Sigmoid():
    @staticmethod
    def f(x):
        return 1.0 / (1.0 + np.exp(-x))
    @staticmethod
    def df(x):
        return Sigmoid.f(x) * (1 - Sigmoid.f(x))

### Loss functions

In [29]:
class MSE():
  @staticmethod
  def f(target, output):
    return np.mean((target - output) **2)
  
  @staticmethod 
  def df(target, output):
    return 2 * (target - output) / np.size(output)

class Crossentropy():
  @staticmethod
  def f(target, output):
    return np.mean(-target * np.log(output) - (1 - target) * np.log(1 - output))
  
  @staticmethod 
  def df(target, output):
    return ((1 -  target) / (1 - output) - target/output) / np.size(target)  

### Dense layer

In [30]:
class Layer:
    """Dit is een dense layer. Alle neuronen zijn fully connected"""
    def __init__(self, neurons, activation=Linear):
        self.neurons = neurons
        self.activation = activation
        self.weights = None
        self.biases = None
        self.lr = 0.02 # Snelheid waarmee de model traint -> staat nu hardcoded

    def initialize(self, prev_neurons):
        """De neuronen van de input layer is gelijk aan de neuronen van de output layer van de layer ervoor"""
        self.weights = np.random.normal(0.0, 1.0, (self.neurons, prev_neurons))
        self.biases = np.random.normal(size=(self.neurons, 1))

    def feed_forward(self, input_array) -> np.ndarray:
        # Input en output opslaan -> zijn nodig voor gradient descent
        self.inputs = input_array 

        # Y = W * I + B (matrix * vector)
        self.output = self.activation.f(np.dot(self.weights, self.inputs) + self.biases) 
        return self.output

    def gradient_descent(self, errors):
        """De waardes voor de minimum worden hier berekend 
        en worden de weights en biases geupdate"""
        
        delta_weights = self.lr * np.dot((errors * self.activation.df(self.output)), self.inputs.T)
        delta_biases = self.lr * errors * self.activation.df(self.output)

        self.weights = np.add(self.weights, delta_weights)
        self.biases = np.add(self.biases, delta_biases)

### Neurale netwerk

In [31]:
class Sequential():
    def __init__(self):
        self.layers = []

    def add(self, layer):
        # Initialiseer de laag met weights en biases alleen als het niet de input laag is.
        if len(self.layers) > 0:
            prev_neurons = self.layers[-1].neurons
            layer.initialize(prev_neurons)
        
        self.layers.append(layer)
        

    def predict(self, input_array: np.ndarray, transposed = False):        
        output = input_array
        
        if not transposed:
            output = output.T

        for lay in self.layers[1:]:
            output = lay.feed_forward(output)
        return output

    def fit(self, input_array: np.ndarray ,output_array: np.ndarray, loss_function=MSE, epochs=10):
        assert isinstance(input_array, np.ndarray)
        assert isinstance(output_array, np.ndarray)

        for epoch in range(epochs):
            sum_errors = 0

            for x, y in zip(input_array, output_array):
                
                # feedforward
                prediction = self.predict(x.reshape(-1,1), transposed = True) # Reshape maakt ndmin 2 en transposed het al

                # houdt errors bij om het te visualiseren
                sum_errors += loss_function.f(y, prediction) 

                # backprop
                error = np.array(loss_function.df(y, prediction), ndmin=2) # gebruik afgeleide om weten of we de gewichten moeten verhogen of verlagen

                for lay in reversed(self.layers[1:]): # We moeten de eerste layer niet hebben. Dat is de input array en bevat eigenlijk geen weights of biases
                    lay.gradient_descent(error) # update
                    error = np.dot(lay.weights.T, error) # bereken error voor de nieuwe layer           
            sum_errors /= len(input_array)

            print(f"Epoch {epoch+1}/{epochs}:")
            print(f"Error: {sum_errors}\n")
        print(f"\nFinished Training\n{'=' * 50}")



## Testing

#### Regressie

Getallen optellen van 3

e.g. array van [0.23, 0.56, 0.14] = 0.93

In [47]:
X_train = np.array([[random() for _ in range(3)] for _ in range(1000)])
y_train = np.array([i[0] + i[1] + i[2] for i in X_train])

X_test = np.array([[3.1, 8.4, 4.0], [50.0, 32.3, 6.6],[0.4, 0.4, 0.12], [0.27, 0.32, 0.02], [0.1, 0.13, 0.32], [0.76, 0.22, 0], [0.26, 0.35, 0.05]]) 
y_test = np.array([[15.5, 88.9, 0.92, 0.61, 0.56, 0.98, 0.66]])

In [59]:
model = Sequential()

model.add(Layer(3, activation=Relu))
model.add(Layer(5, activation=Relu))
model.add(Layer(4, activation=Linear))
model.add(Layer(1, activation=Relu))

In [60]:
model.fit(X_train, y_train, epochs=10, loss_function=MSE)

Epoch 1/10:
Error: 0.07616433984313936

Epoch 2/10:
Error: 0.0017306746463649442

Epoch 3/10:
Error: 0.0006620094085098523

Epoch 4/10:
Error: 0.0003764766923442651

Epoch 5/10:
Error: 0.00023522119671257882

Epoch 6/10:
Error: 0.00015553869266058114

Epoch 7/10:
Error: 0.00011098869630332114

Epoch 8/10:
Error: 8.249381305802736e-05

Epoch 9/10:
Error: 6.433063487185236e-05

Epoch 10/10:
Error: 5.251518377495205e-05


Finished Training


In [61]:
model.predict(np.array([[0.53, 0.11, 0.23]]))

array([[0.87121684]])

In [62]:
pred = model.predict(X_test)
pred = np.around(pred, 2)
pred

array([[11.22, 77.24,  0.92,  0.61,  0.55,  0.98,  0.66]])

In [63]:
r2_score(y_test.flatten(), pred.flatten())

0.9761778890175679

### Binary classification

In [64]:
def get_model() -> Sequential:
    model = Sequential()

    model.add(Layer(2, activation=Relu))
    model.add(Layer(5, activation=Relu))
    model.add(Layer(3, activation=Relu))
    model.add(Layer(1, activation=Sigmoid))

    return model


#### AND

In [65]:
X_train = np.array([[0,0], [0,1], [1,0], [1,1]])
y_train = np.array([0,0,0,1])

X_test = np.array([[1,0], [1,1], [0, 0]])
y_test = np.array([0,1,0])

In [66]:
model = get_model()

model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000:
Error: 0.2536117474496592

Epoch 2/1000:
Error: 0.2421937276216269

Epoch 3/1000:
Error: 0.2328761303479175

Epoch 4/1000:
Error: 0.2332282635445474

Epoch 5/1000:
Error: 0.22680996261653855

Epoch 6/1000:
Error: 0.2268984872437093

Epoch 7/1000:
Error: 0.22190458518369371

Epoch 8/1000:
Error: 0.22028290683289747

Epoch 9/1000:
Error: 0.21836326718614457

Epoch 10/1000:
Error: 0.2142741327886002

Epoch 11/1000:
Error: 0.2156718389148684

Epoch 12/1000:
Error: 0.21064695326986832

Epoch 13/1000:
Error: 0.21043476424883661

Epoch 14/1000:
Error: 0.20945096369611477

Epoch 15/1000:
Error: 0.20550281404040294

Epoch 16/1000:
Error: 0.20483629686889274

Epoch 17/1000:
Error: 0.20483682075451937

Epoch 18/1000:
Error: 0.1995963596913202

Epoch 19/1000:
Error: 0.19892363249440792

Epoch 20/1000:
Error: 0.19184129494049101

Epoch 21/1000:
Error: 0.18726471476610568

Epoch 22/1000:
Error: 0.18269199110052387

Epoch 23/1000:
Error: 0.17904985369767146

Epoch 24/1000:
Error: 0.1776

In [67]:
pred = model.predict(X_test)
print(f'prediction: {pred}')
pred = np.around(pred)
print(f"Prediction rounded: {pred}")

prediction: [[1.41228712e-03 9.97431546e-01 5.56439255e-07]]
Prediction rounded: [[0. 1. 0.]]


In [68]:
accuracy_score(y_test, pred.flatten())

1.0

#### OR

In [83]:
y_train = np.array([0,1,1,1])
y_test = np.array([1,1,0])

model = get_model()
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000:
Error: 0.3217551270485466

Epoch 2/1000:
Error: 0.31244325748835355

Epoch 3/1000:
Error: 0.30415012476620995

Epoch 4/1000:
Error: 0.2975718891329566

Epoch 5/1000:
Error: 0.2901345552489365

Epoch 6/1000:
Error: 0.2854768076614517

Epoch 7/1000:
Error: 0.24358558136490807

Epoch 8/1000:
Error: 0.20146759894806657

Epoch 9/1000:
Error: 0.19287644047309543

Epoch 10/1000:
Error: 0.18829922703030666

Epoch 11/1000:
Error: 0.18269549427743317

Epoch 12/1000:
Error: 0.1776669006498186

Epoch 13/1000:
Error: 0.17250547066400518

Epoch 14/1000:
Error: 0.16712400596468824

Epoch 15/1000:
Error: 0.16154395546945696

Epoch 16/1000:
Error: 0.15579117078722438

Epoch 17/1000:
Error: 0.14989371687544698

Epoch 18/1000:
Error: 0.14401089945973453

Epoch 19/1000:
Error: 0.1378864226419919

Epoch 20/1000:
Error: 0.13175230599183407

Epoch 21/1000:
Error: 0.12557881644339644

Epoch 22/1000:
Error: 0.11940107397982448

Epoch 23/1000:
Error: 0.11325145380293737

Epoch 24/1000:
Error: 0.10

In [84]:
pred = model.predict(X_test)
print(f'prediction: {pred}')
pred = np.around(pred)
print(f"Prediction rounded: {pred}")

prediction: [[0.99962301 0.99999983 0.11316019]]
Prediction rounded: [[1. 1. 0.]]


In [85]:
accuracy_score(y_test, pred.flatten())

1.0

#### XOR

Dit keer met wat minder epochs en minder layers om verschilt te zien

In [86]:
y_train = np.array([0, 1,1,0])
y_test = np.array([1,0,0])
model = get_model()

model.fit(X_train, y_train, epochs=150)

Epoch 1/150:
Error: 0.2675810933913045

Epoch 2/150:
Error: 0.2672083661856584

Epoch 3/150:
Error: 0.2670021315979473

Epoch 4/150:
Error: 0.2669579729407278

Epoch 5/150:
Error: 0.26706857587309424

Epoch 6/150:
Error: 0.2673240026145539

Epoch 7/150:
Error: 0.26771207375746514

Epoch 8/150:
Error: 0.26821882258192553

Epoch 9/150:
Error: 0.26882898475294714

Epoch 10/150:
Error: 0.26952648877929924

Epoch 11/150:
Error: 0.27029491853944193

Epoch 12/150:
Error: 0.2711179270616938

Epoch 13/150:
Error: 0.2719795891041363

Epoch 14/150:
Error: 0.2728646877083053

Epoch 15/150:
Error: 0.27375893601410933

Epoch 16/150:
Error: 0.27427558496169707

Epoch 17/150:
Error: 0.2742241469603575

Epoch 18/150:
Error: 0.27436908957974804

Epoch 19/150:
Error: 0.2746349210318767

Epoch 20/150:
Error: 0.27440672044672565

Epoch 21/150:
Error: 0.2748105965530662

Epoch 22/150:
Error: 0.274530772784577

Epoch 23/150:
Error: 0.27477261092808514

Epoch 24/150:
Error: 0.2748019769338894

Epoch 25/150:
E

In [87]:
pred = model.predict(X_test)
print(f'prediction: {pred}')
pred = np.around(pred)
print(f"Prediction rounded: {pred}")

prediction: [[0.68702029 0.52400776 0.45839316]]
Prediction rounded: [[1. 1. 0.]]


In [88]:
accuracy_score(y_test, pred.flatten())

0.6666666666666666

### Evaluatie

De Sequential model hierboven is een simpele neurale netwerk. Niet al te best natuurlijk, maar het is een begin. Het werkt wel degelijk. De softmax functie heb ik niet werkend gekregen. Daarom is alleen een regressie en binary classification voorbeeld laten zien.